In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import os
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
wandb.init(project="cifar")
config = wandb.config
config.dropout = 0.25
config.dense_layer_nodes = 100
config.batch_size = 32
config.epochs = 50

class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
num_classes = len(class_names)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

W&B Run: https://app.wandb.ai/l2k2/cifar/runs/k9q4itov
Call `%%wandb` in the cell containing your training loop to display live results.


In [3]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [28]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))

model.add(Flatten())
model.add(Dense(config.dense_layer_nodes, activation='relu'))
model.add(Dropout(config.dropout))
model.add(Dense(num_classes, activation='softmax'))

In [29]:
learn_rate = 0.004
batch_size = 128
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learn_rate),
              metrics=['accuracy'])


In [30]:
wandb.init()
model.fit(X_train,y_train,batch_size=batch_size,
                        epochs=300,
                        validation_data=(X_test, y_test),
                        callbacks=[WandbCallback(),ReduceLROnPlateau()]
)

W&B Run: https://app.wandb.ai/l2k2/cifar/runs/snh4qfgk
Call `%%wandb` in the cell containing your training loop to display live results.
Train on 50000 samples, validate on 10000 samples
Epoch 1/300
50000/50000 [==============================] - 6s 127us/step - loss: 1.7099 - acc: 0.3735 - val_loss: 1.4350 - val_acc: 0.4841
Epoch 2/300
50000/50000 [==============================] - 5s 106us/step - loss: 1.4578 - acc: 0.4704 - val_loss: 1.3006 - val_acc: 0.5401
Epoch 3/300
50000/50000 [==============================] - 5s 102us/step - loss: 1.3725 - acc: 0.5022 - val_loss: 1.2329 - val_acc: 0.5600
Epoch 4/300
50000/50000 [==============================] - 5s 103us/step - loss: 1.3162 - acc: 0.5239 - val_loss: 1.2082 - val_acc: 0.5682
Epoch 5/300
50000/50000 [==============================] - 5s 102us/step - loss: 1.2727 - acc: 0.5380 - val_loss: 1.1415 - val_acc: 0.5951
Epoch 6/300
50000/50000 [==============================] - 5s 103us/step - loss: 1.2365 - acc: 0.5553 - val_loss: 1.18